In [ ]:
import hmftpy as hmf
from hmftpy.plaquettes.triangular import plaq12, plaq12z, plaq9p, plaq9d, plaq3
from hmftpy.operators import mf_ops, inner_hamiltonian, periodic_hamiltonian
from hmftpy import do_hmft
from hmftpy.hmf import get_mfs
from hmftpy.plot_spins import draw_tri_lattice, draw_cluster, plot_spins
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

plaq = plaq12
L = plaq['L']
basis = spin_basis_1d(L, pauli=0, Nup=(5,6,7))

ops = hmf.operators.mf_ops(plaq, basis)
maxit=50

def ni_op(plaq, basis):
    A, B, C = plaq['n_nearest_sublattice']
    a_op = quantum_operator({'static': [['x', [[1/len(A), j] for j in A]]]}, basis=basis, check_symm=False, check_herm=False)
    b_op = quantum_operator({'static': [['x', [[-0.5/len(B), j] for j in B]],
                                        ['y', [[0.5*np.sqrt(3)/len(B), j] for j in B]]]}, basis=basis, check_symm=False, check_herm=False)
    c_op = quantum_operator({'static': [['x', [[-0.5/len(C), j] for j in C]],
                                        ['y', [[-0.5*np.sqrt(3)/len(C), j] for j in C]]]}, basis=basis, check_symm=False, check_herm=False)
    return a_op + b_op + c_op

m_op = ni_op(plaq, basis)

In [ ]:
steps = 30
l = 9
alphas = np.linspace(0.4, .6, l)
print(alphas)
energies = np.zeros((l, steps))
Nis = np.zeros((l, steps))
Ni2s = np.zeros((l, steps))

J2s = np.linspace(-.1, 0.1, steps)
mf0s = [None for i in range(l)]

def couplings(J2, alpha=0):
    interactions = {'local': {},
                    'nearest': {'xx': 1, 'yy': 1, 'zz': alpha},
                    'n_nearest': {'xx': J2, 'yy': J2, 'zz': alpha*J2},
                    'n_n_nearest': {}}
    return interactions

In [ ]:
for i, J2 in enumerate(tqdm(J2s, ascii=True)):
    for j, alpha in enumerate(alphas):
        interactions = couplings(J2, alpha)
        energies[j,i], v, mf0s[j], cvg = do_hmft(plaq, interactions, basis, 
                                                max_iter=maxit, mf0=mf0s[j],
                                                ops=ops,
                                                mf_cvg=False, every_other=True)
        Nis[j,i] = np.real(m_op.matrix_ele(v, v))
        Ni2s[j,i] = np.real(m_op.matrix_ele(v, m_op.dot(v)))

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
for j, alpha in enumerate(alphas):
    if 0.1 < alpha < 0.8:
        plt.plot(J2s, np.gradient(energies[j,:], J2s), label=np.round(alpha, 2))
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(4,3), dpi=200)
for j, alpha in enumerate(alphas):
    if 0.1 < alpha < 0.8:
        plt.plot(J2s, np.abs(Nis[j,:]), label=np.round(alpha, 1))
plt.legend()
plt.show()